In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-08-29"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
292,2024-08-29,Vietnã Vba,09:30,Can Tho Catfish,Hanoi Buffaloes,1.67,2.02,157.5,1.82,1.84,-3.5,2.04,1.66,6RMOHwUp,0.598802,0.495050,0.549451,0.543478,0.093852,147.756,53.612704,0.362846,1.8,1.643168,0.912871,148.68,1.888,0.597972,0.316722,4.0,158.618,36.724163,0.231526,3.0,0.000000,0.000000,170.10,1.738,0.613572,0.353033,65.0,84,81,1.77,2.10,0.000,156.900,0.134139,0.007728,0.145244,0.00,0.000,inf,0.000000,0.0,0.000000,-0.19,-0.038,-26.842105,0.000000,0.0,0.000000
293,2024-08-29,Filipinas Copa Dos Governadores,08:30,TNT Tropang Giga,Magnolia Hotshots,1.77,1.99,192.5,1.86,1.80,-2.5,2.01,1.72,EeAj7WaT,0.564972,0.502513,0.537634,0.555556,0.067484,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,95,105,1.25,1.28,0.000,0.000,0.082747,0.023184,0.109952,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
294,2024-08-29,Mundo Amistoso Interclubes,11:30,OGM Ormanspor (Tur),Gaziantep (Tur),1.72,1.95,160.5,1.83,1.83,-2.5,1.99,1.70,I1Gpygj2,0.581395,0.512821,0.546448,0.546448,0.094216,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.088629,0.000000,0.111144,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
295,2024-08-29,Paraguai Lnb,20:30,Dep. San Jose,Olimpia,1.09,5.61,142.5,1.84,1.82,-13.5,1.95,1.72,fu11bWfc,0.917431,0.178253,0.543478,0.549451,0.095684,107.666,13.460761,0.125023,2.4,1.341641,0.559017,101.00,1.348,0.304417,0.225829,84.0,138.904,25.199484,0.181417,1.8,1.643168,0.912871,114.24,1.996,0.512523,0.256775,3.0,100,84,1.01,1.36,110.398,138.966,0.954066,0.007728,0.088629,0.46,0.092,0.978261,0.775710,0.9,0.124290,0.17,0.034,135.588235,0.538156,0.7,0.161844
296,2024-08-29,Paraguai Lnb,21:30,Ciudad Nueva,Felix Perez Cardozo,3.35,1.26,147.5,1.85,1.81,3.5,2.01,1.68,0x49dAP9,0.298507,0.793651,0.540541,0.552486,0.092158,335.484,286.180930,0.853039,1.2,1.643168,1.369306,397.50,5.650,5.404313,0.956516,-40.0,257.432,176.270670,0.684727,0.6,1.341641,2.236068,111.60,3.834,3.228735,0.842132,-29.0,53,93,7.50,1.20,468.448,441.230,0.641151,0.015456,0.126474,-2.69,-0.538,-4.368030,0.000000,0.0,0.000000,-5.00,-1.000,-0.260000,0.000000,0.0,0.000000
297,2024-08-29,Singapura Nbl,08:00,Siglap,Eng Tat Hornets,2.55,1.49,137.5,1.83,1.83,6.5,1.83,1.83,jy0lqgnm,0.392157,0.671141,0.546448,0.546448,0.063298,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,95,0.00,1.83,0.000,0.000,0.371056,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
298,2024-08-29,Singapura Nbl,09:30,Police,Nanyang,1.49,2.55,130.5,1.83,1.83,-4.5,1.83,1.83,hU3IUKKP,0.671141,0.392157,0.546448,0.546448,0.063298,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,60,48,12.80,12.10,0.000,0.000,0.371056,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
299,2024-08-29,Venezuela Superliga,20:00,Gladiadores,

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
292,09:30,Vietnã Vba,Can Tho Catfish,Hanoi Buffaloes,1.67,Back Home
